In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

# Pending

In [ ]:
# from scipy.io import loadmat

# link = r"D:\external_data\Experiment4"
# filename = r"\Dataset_PWR_WiFi.mat"
# directory = link + filename

# mat = loadmat(directory)

In [ ]:
# # import experimental dataset 2
# folderpath2 = "D:/external_data/Experiment3/csv_files/exp_2"  # CHANGE THIS IF THE PATH CHANGED
# df_exp2 = import_clean_data('exp2',folderpath2)

# lab

### visualisation

In [ ]:
# Visualise spectrogram
img = img.transpose()
img = img[:,::10]
plt.figure(figsize = (15,50))
plt.imshow(img,cmap='jet')
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from data.process_data import stacking
import matplotlib.pyplot as plt



def visual_spectrogram(img,title='spectrogram'):
    
    if len(img.shape) == 3:
        
        img.squeeze()
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(img)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
    plt.figure(figsize = (15,50))
    plt.imshow(img,cmap='jet')
    plt.title(title)
    plt.show()
    
    return 

In [ ]:
import torchvision.models as models
        
#visualize weights for alexnet - first conv layer
alexnet = models.alexnet(pretrained=False)

# Store all conv layer in alexnet
k = []
for m in alexnet.modules():
    if isinstance(m, nn.Conv2d):
        print(m.weight.data.shape)
        k.append(m)
    else:  
        continue


In [ ]:
def visual_single_filter(weight):
    
    img = MinMaxScaler(feature_range=(0,1)).fit_transform(weight)
    
#     img = img.reshape(*img.shape[:-1],1)
    
#     img = np.concatenate((img,img,img),axis=2)
    
#     plt.figure(figsize = (15,10))
    plt.imshow(weight,cmap='jet')
#     plt.show()
    
    return 

def visualise_filters(layer,figsize=(20,15)):
    
    weights = layer.weight.data

    if weights.shape[0] > weights.shape[1]:
        
        weights = weights.transpose(1,0)
        
    rows = weights.shape[0]
    
    cols = weights.shape[1]
    
    axes = []
    
    fig= plt.figure()

    for n in range(rows):

        for c in range(cols):
            
            axes.append(fig.add_subplot(rows, cols, n*cols+c+1))

            single_filter = weights[n,c]
        
            visual_single_filter(single_filter)
            
#     fig.tight_layout()    
    
    plt.show()     
    
    return 

visualise_filters(example)

### CNN-LSTM

### model factory

In [1]:
import time
from time import gmtime, strftime
# a = time.time()
# model  = train(model, train_loader, criterion, optimizer, 48)
# print(time.time() - a)

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchsummary import summary
import torchvision

In [4]:
np.random.seed(1024)
torch.manual_seed(1024)
torch.set_deterministic(True)

In [ ]:
# model 
class model(nn.Module):
    
    def __init__(self):
        super(model, self).__init__()
        pass
    
    def forward(self,x):
        return x

In [ ]:
class PreTraining(nn.Module):
    
    def __init__(self):
        super(PreTraining, self).__init__()
        pass
    
    def forward(self,x):
        return x

In [ ]:
assert False

# Test

In [1]:
from data import process_data
from data.import_data import import_experimental_data
# from data.process_data import DatasetObject
from models.train import * 


C:\ProgramData\Anaconda3\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
# 1. Import data
folderpath1 = "E:/external_data/Experiment3/csv_files/exp_1"
df_exp1 = import_experimental_data(folderpath1)

# 2. Process data
X_ls, y_ls = seperate_dataframes(df_exp1)
del df_exp1

# 3. DatasetObject
exp_1 = create_datasetobject(X_ls, y_ls)
del X_ls,y_ls

# 4. Load data into dataloaders
idxs = [0]
(X_train, y_train,_),(X_test, y_test,_) = exp_1(idxs,return_train_sets=True)
train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test)
del X_train, y_train, X_test, y_test

Found 80 files.
input_user10_bendfwd.csv annotation_user10_bendfwd.csv user10
input_user10_kneel.csv annotation_user10_kneel.csv user10
input_user10_lie.csv annotation_user10_lie.csv user10
input_user10_sit.csv annotation_user10_sit.csv user10
input_user10_sitrotate.csv annotation_user10_sitrotate.csv user10
input_user10_stand.csv annotation_user10_stand.csv user10
input_user10_standrotate.csv annotation_user10_standrotate.csv user10
input_user10_walking.csv annotation_user10_walking.csv user10
input_user1_bendfwd.csv annotation_user1_bendfwd.csv user1
input_user1_kneel.csv annotation_user1_kneel.csv user1
input_user1_lie.csv annotation_user1_lie.csv user1
input_user1_sit.csv annotation_user1_sit.csv user1
input_user1_sitrotate.csv annotation_user1_sitrotate.csv user1
input_user1_stand.csv annotation_user1_stand.csv user1
input_user1_standrotate.csv annotation_user1_standrotate.csv user1
input_user1_walking.csv annotation_user1_walking.csv user1
input_user2_bendfwd.csv annotation_user2

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
criterion = nn.MSELoss()
epochs = 500
model  = train_unsupervised(model=model, train_loader=train_loader, criterion=criterion, 
                 optimizer=optimizer, end=epochs, start=105, evaluation=None, auto_save=True,
                 name='AEPretrain_module',test_loader=None)

Start Training
Epoch 105: >>>>>>>>>>>>> loss: 404.9288024902344
Epoch 106: >>>>>>>>>>>>> loss: 380.29608154296875
Epoch 107: >>>>>>>>>>>>> loss: 346.3295593261719
Epoch 108: >>>>>>>>>>>>> loss: 404.4480895996094
Epoch 109: >>>>>>>>>>>>> loss: 374.84027099609375
Epoch 110: >>>>>>>>>>>>> loss: 394.3640441894531
Epoch 111: >>>>>>>>>>>>> loss: 395.0081787109375
Epoch 112: >>>>>>>>>>>>> loss: 384.3662109375
Epoch 113: >>>>>>>>>>>>> loss: 419.6768798828125
Epoch 114: >>>>>>>>>>>>> loss: 382.2357482910156
Epoch 115: >>>>>>>>>>>>> loss: 400.4671630859375
Epoch 116: >>>>>>>>>>>>> loss: 419.6222229003906
Epoch 117: >>>>>>>>>>>>> loss: 572.9689331054688
Epoch 118: >>>>>>>>>>>>> loss: 551.9775390625
Epoch 119: >>>>>>>>>>>>> loss: 472.08685302734375
Epoch 120: >>>>>>>>>>>>> loss: 459.9753723144531
Epoch 121: >>>>>>>>>>>>> loss: 428.83074951171875
Epoch 122: >>>>>>>>>>>>> loss: 387.3523864746094
Epoch 123: >>>>>>>>>>>>> loss: 404.9145812988281
Epoch 124: >>>>>>>>>>>>> loss: 368.30902099609375
Epoch 

Epoch 270: >>>>>>>>>>>>> loss: 397.2841796875
Epoch 271: >>>>>>>>>>>>> loss: 364.1202392578125
Epoch 272: >>>>>>>>>>>>> loss: 353.5174560546875
Epoch 273: >>>>>>>>>>>>> loss: 372.90618896484375
Epoch 274: >>>>>>>>>>>>> loss: 366.4631652832031
Epoch 275: >>>>>>>>>>>>> loss: 385.9159851074219
Epoch 276: >>>>>>>>>>>>> loss: 389.7668762207031
Epoch 277: >>>>>>>>>>>>> loss: 363.4786071777344
Epoch 278: >>>>>>>>>>>>> loss: 399.7900390625
Epoch 279: >>>>>>>>>>>>> loss: 356.1817626953125
Epoch 280: >>>>>>>>>>>>> loss: 380.22418212890625
Epoch 281: >>>>>>>>>>>>> loss: 359.0148010253906
Epoch 282: >>>>>>>>>>>>> loss: 334.2544250488281
Epoch 283: >>>>>>>>>>>>> loss: 343.4923095703125
Epoch 284: >>>>>>>>>>>>> loss: 350.2142639160156
Epoch 285: >>>>>>>>>>>>> loss: 374.9696960449219
Epoch 286: >>>>>>>>>>>>> loss: 377.9765319824219
Epoch 287: >>>>>>>>>>>>> loss: 355.0858154296875
Epoch 288: >>>>>>>>>>>>> loss: 368.6839904785156
Epoch 289: >>>>>>>>>>>>> loss: 359.736328125
Epoch 290: >>>>>>>>>>>>> los

Epoch 434: >>>>>>>>>>>>> loss: 340.6492614746094
Epoch 435: >>>>>>>>>>>>> loss: 357.8119201660156
Epoch 436: >>>>>>>>>>>>> loss: 341.4598083496094
Epoch 437: >>>>>>>>>>>>> loss: 351.6108093261719
Epoch 438: >>>>>>>>>>>>> loss: 394.90484619140625
Epoch 439: >>>>>>>>>>>>> loss: 375.1571044921875
Epoch 440: >>>>>>>>>>>>> loss: 364.2898864746094
Epoch 441: >>>>>>>>>>>>> loss: 337.50067138671875
Epoch 442: >>>>>>>>>>>>> loss: 380.54168701171875
Epoch 443: >>>>>>>>>>>>> loss: 365.7223205566406
Epoch 444: >>>>>>>>>>>>> loss: 347.37860107421875
Epoch 445: >>>>>>>>>>>>> loss: 336.8121032714844
Epoch 446: >>>>>>>>>>>>> loss: 338.69183349609375
Epoch 447: >>>>>>>>>>>>> loss: 348.83441162109375
Epoch 448: >>>>>>>>>>>>> loss: 367.85302734375
Epoch 449: >>>>>>>>>>>>> loss: 393.9852294921875
Epoch 450: >>>>>>>>>>>>> loss: 362.5007019042969
Epoch 451: >>>>>>>>>>>>> loss: 356.92596435546875
Epoch 452: >>>>>>>>>>>>> loss: 364.88897705078125
Epoch 453: >>>>>>>>>>>>> loss: 355.4370422363281
Epoch 454: >>>

In [8]:
mdl = Autoencoder(Encoder(),Decoder())
criterion, optimizer = setting(mdl)
mdl,_ = load_checkpoint(mdl,optimizer,'./models/saved_models/AEPretrain_module_checkpoint_500__2021_01_13_15_56')
encoder = mdl.encoder
cnn = CNN_module(encoder,Classifier(1024))

In [9]:
criterion, optimizer = setting(cnn)

In [16]:
epochs = 500
model  = train(model=cnn, train_loader=train_loader, criterion=criterion, 
                 optimizer=optimizer, end=epochs, start=100, evaluation=True, auto_save=True,
                 name='AEPretrain_baseline',test_loader=test_loader)

Start Training
Epoch 100: >>>>>>>>>>>>> loss: 0.01584584452211857
              precision    recall  f1-score   support

           0       0.82      0.36      0.50        25
           1       1.00      0.92      0.96        25
           2       1.00      0.36      0.53        25
           3       0.38      0.48      0.42        25
           4       0.72      0.52      0.60        25
           5       0.50      0.60      0.55        25
           6       0.56      1.00      0.71        25
           7       0.12      0.16      0.14        25

    accuracy                           0.55       200
   macro avg       0.64      0.55      0.55       200
weighted avg       0.64      0.55      0.55       200

Evaluation:
[[ 9  0  0  0  0  6  0 10]
 [ 0 23  0  2  0  0  0  0]
 [ 2  0  9 11  0  0  0  3]
 [ 0  0  0 12  0  7  0  6]
 [ 0  0  0  2 13  2  0  8]
 [ 0  0  0  4  5 15  0  1]
 [ 0  0  0  0  0  0 25  0]
 [ 0  0  0  1  0  0 20  4]]
save checkpoint in : ./models/saved_models/AEPretrain_

Epoch 357: >>>>>>>>>>>>> loss: 0.00026589646586216986
Epoch 358: >>>>>>>>>>>>> loss: 8.294134204334114e-06
Epoch 359: >>>>>>>>>>>>> loss: 1.5823177818674594e-05
Epoch 360: >>>>>>>>>>>>> loss: 0.0011745686642825603
Epoch 361: >>>>>>>>>>>>> loss: 0.011049394495785236
Epoch 362: >>>>>>>>>>>>> loss: 4.748464561998844e-05
Epoch 363: >>>>>>>>>>>>> loss: 0.0007144450792111456
Epoch 364: >>>>>>>>>>>>> loss: 0.000516003929078579
Epoch 365: >>>>>>>>>>>>> loss: 0.0056405263021588326
Epoch 366: >>>>>>>>>>>>> loss: 3.605617530411109e-05
Epoch 367: >>>>>>>>>>>>> loss: 0.00028921058401465416
Epoch 368: >>>>>>>>>>>>> loss: 0.00016224849969148636
Epoch 369: >>>>>>>>>>>>> loss: 0.0004348265356384218
Epoch 370: >>>>>>>>>>>>> loss: 0.0026760075706988573
Epoch 371: >>>>>>>>>>>>> loss: 0.00015237713523674756
Epoch 372: >>>>>>>>>>>>> loss: 0.00034413617686368525
Epoch 373: >>>>>>>>>>>>> loss: 3.2275223929900676e-05
Epoch 374: >>>>>>>>>>>>> loss: 7.840039324946702e-05
Epoch 375: >>>>>>>>>>>>> loss: 0.00011984

In [34]:
from models.self_supervised import *
from models.CNN import Encoder

def build_simclr():
    encoder = Encoder()
    dataug = DataAugmentation(size=(1000,90))
    n_features = [1024,128,8]
    simclr = SimCLR(encoder=encoder, n_features=n_features, transformer=dataug)
    return simclr,dataug

def setting_(model):
    criterion = NT_Xent(batch_size=128,temperature=0.1)
    optimizer = torch.optim.Adam(list(model.parameters()), lr=0.003)
    return criterion, optimizer

def train_unsupervised(model, train_loader, criterion, optimizer, epochs, auto_save = False):
    print('Start Training')
    for i in range(epochs):
        print(f'epoch: {i+1},  batch:', end='')
        for b, (X,_) in enumerate(train_loader):
            print(">",end='')
            optimizer.zero_grad()
            z1,z2 = model(X)
            loss = criterion(z1,z2)
            loss.backward()
            optimizer.step()
            
        print(f' loss: {loss.tolist()}')
        
        if (i+1)%100 == 0 and auto_save == True:
            
            directory = make_directory('SimCLR_training', epoch=i+1, filepath='./models/saved_models/')
            save_checkpoint(model,optimizer,i+1,directory)

    return model

simclr, dataug = build_simclr()
sample = torch.rand(size=(5,1,100,90))
simclr.forward(sample)
summary(simclr,sample.shape[1:])
# criterion, optimizer = setting_(model)
# epochs = 500
# simclr = train_unsupervised(model=simclr, 
#                            train_loader=train_loader, 
#                            criterion=criterion, 
#                            optimizer=optimizer, 
#                            epochs=epochs,
#                            auto_save=True)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
  DataAugmentation-1  [[-1, 1, 1000, 90], [-1, 1, 1000, 90]]               0
            Conv2d-2          [-1, 64, 200, 18]           1,664
              ReLU-3          [-1, 64, 200, 18]               0
            Lambda-4          [-1, 64, 200, 18]               0
         MaxPool2d-5          [-1, 64, 100, 17]               0
            Conv2d-6           [-1, 128, 33, 5]          73,856
              ReLU-7           [-1, 128, 33, 5]               0
            Lambda-8           [-1, 128, 33, 5]               0
         MaxPool2d-9           [-1, 128, 16, 4]               0
           Conv2d-10            [-1, 256, 8, 2]         131,328
             ReLU-11            [-1, 256, 8, 2]               0
           Lambda-12            [-1, 256, 8, 2]               0
        MaxPool2d-13            [-1, 256, 4, 1]               0
          Encoder-14      

### CNN_baseline 

In [ ]:
torch.manual_seed(0)
np.random.seed(0)
torch.set_deterministic(True)